We continue to work with the knots of at most 9 crossings that have hyperbolic 0-surgeries. We go through Burton's list and compare the volumes.

First we load the necessary code:

In [1]:
import snappy
import csv
import time

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=1,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def change_notation(dt_code):
    """
    Changes Dowker-Thistlewait notation from alphabetical to numerical
    Input:
        dt_code (string): alphabetical DT notation
    Return:
        (string): numerical DT notation
    """
    alpha = "abcdefghijklmnopqrstuvwxyz"
    Alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    result = []
    for letter in dt_code:
        if letter in alpha:
            result.append(2* (alpha.index(letter) + 1))
        elif letter in Alpha:
            result.append(-2 * (Alpha.index(letter) + 1))
        else:
            print(dt_code)
    return "DT: " + str([tuple(result)])



Next, we load the lists:

In [2]:
knots_with_hyperbolic_0_surgery=['6_3','6_2','7_7','7_6','7_5','7_3','8_14','9_8','8_15','8_10','8_8','8_12','8_7','8_13','8_2','8_11','8_6','8_18','9_24','8_5','8_17','8_16','8_9','8_4','8_21','9_30','9_22','9_19','9_25','9_28','9_29','9_32','9_36','9_15','9_33','9_27','9_31','9_17','9_26','9_23','9_14','9_37','9_20','9_11','9_21','9_12','9_6','9_18','9_16','9_7','9_34','9_41','9_38','9_39','9_9','9_13','9_4','9_40','9_3','9_10','9_44','9_45','9_43','9_42','9_48','9_47','9_49']
len(knots_with_hyperbolic_0_surgery)

67

In [3]:
start_time = time.time()

pairs_with_same_volume=[]
pairs_with_unclear_volume=[]

doubles_first=['8_14','8_18','9_28']
doubles_second=['9_8','9_24','9_29']

for knot in knots_with_hyperbolic_0_surgery:
    knots_with_same_alex=[] 
    if knot in doubles_first:
        with open(knot+'.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                knots_with_same_alex.append(row)
            knots_with_same_alex=knots_with_same_alex[3:] 
    if knot in doubles_second:
        if knot=='9_8':
            first='8_14'
        if knot=='9_24':
            first='8_18'
        if knot=='9_29':
            first='9_28'    
        with open(first+'.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                knots_with_same_alex.append(row)
            knots_with_same_alex=knots_with_same_alex[3:] 
    if knot not in doubles_first+doubles_second:
        with open(knot+'.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                knots_with_same_alex.append(row)
            knots_with_same_alex=knots_with_same_alex[2:] 
    print('We are checking the knots with the same Alexander polynomial as:',knot)
    
    K=snappy.Manifold(knot)
    K.dehn_fill((0,1))
    vol0=better_volume(K,try_hard=True)
    print('Volume of knot:',vol0)
    
    print('Number of knots we need to check:',len(knots_with_same_alex))
    
    for [knot1,cros,DT] in knots_with_same_alex:
        K1=snappy.Link(change_notation(DT)).exterior()
        K1.dehn_fill((0,1))
        vol=better_volume(K1,try_hard=True)
        if vol==0:           
            print('We could not determine the hyperbolicity of:',knot1,DT)
            pairs_with_unclear_volume.append([knot,knot1,cros,DT])
        if vol!=0:
            if vol.overlaps(vol0):
                pairs_with_same_volume.append([knot,knot1,cros,DT])
                print('We found a knot with overlapping volume:',knot1,DT)

    print('----------------')
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))

We are checking the knots with the same Alexander polynomial as: 6_3
Volume of knot: 4.059766425639?
Number of knots we need to check: 16827
We found a knot with overlapping volume: 19nh_000000078 khPiOjNRcesGqlFDBma
----------------
We are checking the knots with the same Alexander polynomial as: 6_2
Volume of knot: 3.770829451108?
Number of knots we need to check: 18651
We could not determine the hyperbolicity of: 19ns_050 mQkGNODJrLcHaPEFSiB
----------------
We are checking the knots with the same Alexander polynomial as: 7_7
Volume of knot: 6.332666642499?
Number of knots we need to check: 2939
----------------
We are checking the knots with the same Alexander polynomial as: 7_6
Volume of knot: 6.180274419374?
Number of knots we need to check: 5264
----------------
We are checking the knots with the same Alexander polynomial as: 7_5
Volume of knot: 5.987810443355?
Number of knots we need to check: 2836
----------------
We are checking the knots with the same Alexander polynomial as

----------------
We are checking the knots with the same Alexander polynomial as: 9_34
Volume of knot: 13.23190917916?
Number of knots we need to check: 155
----------------
We are checking the knots with the same Alexander polynomial as: 9_41
Volume of knot: 10.85352845425?
Number of knots we need to check: 229
----------------
We are checking the knots with the same Alexander polynomial as: 9_38
Volume of knot: 12.1931276781?
Number of knots we need to check: 27
----------------
We are checking the knots with the same Alexander polynomial as: 9_39
Volume of knot: 11.65700086215?
Number of knots we need to check: 71
----------------
We are checking the knots with the same Alexander polynomial as: 9_9
Volume of knot: 7.82241098274?
Number of knots we need to check: 25
----------------
We are checking the knots with the same Alexander polynomial as: 9_13
Volume of knot: 8.44017037566?
Number of knots we need to check: 22
----------------
We are checking the knots with the same Alexander

In [5]:
pairs_with_unclear_volume

[['6_2', '19ns_050', '19', 'mQkGNODJrLcHaPEFSiB'],
 ['7_3', '18ns_08', '18', 'lPjFNMIqKcGaODERhB'],
 ['8_14', '17ns_28', '17', 'jOhLGpIcEaMNDKQfB'],
 ['9_8', '17ns_28', '17', 'jOhLGpIcEaMNDKQfB'],
 ['8_4', '17ns_14', '17', 'kOiLMHpJcFaNDEQgB'],
 ['9_4', '16ns_02', '16', 'kNiLMHoJcFaEDPgB']]

In [8]:
for x in pairs_with_unclear_volume:
    K=snappy.Link(change_notation(x[3])).exterior()
    print(K.splitting_surfaces())

[Orientable two-sided with euler = 0]
[Orientable two-sided with euler = 0]
[Orientable two-sided with euler = 0]
[Orientable two-sided with euler = 0]
[Orientable two-sided with euler = 0]
[Orientable two-sided with euler = 0]


From the Burton notation we can read-off that they are all sattelites and thus their 0-surgeries are also not hyperbolic.

In [9]:
pairs_with_same_volume

[['6_3', '19nh_000000078', '19', 'khPiOjNRcesGqlFDBma']]

In [10]:
K=snappy.Manifold('6_3')
K.dehn_fill((0,1))
F=snappy.Link(change_notation('khPiOjNRcesGqlFDBma')).exterior()
F.dehn_fill((0,1))

In [11]:
K.is_isometric_to(F)

True

We check the cirality of this example as well:

In [12]:
def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

### CHECKING FOR ISOMETRIES

def better_is_isometric_to(X,Y,return_isometries=False,index=100,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies.
     """
    if return_isometries==False:
        w='unclear'
        if X.homology()!=Y.homology():
            return False
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w==True:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=1,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=1,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,index=100,try_hard=False)
                    if w==True:
                        return w
        return 'unclear'
    if return_isometries==True:
        w=False
        if X.homology()!=Y.homology():
            return []
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y,return_isometries)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w!=False:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=10,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=10,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,return_isometries,index=100,try_hard=False)
                    if w!=False:
                        return w
        return []
   
def orientation_preserving_isometric(M,N,index=100,try_hard=False):
    '''
    Searches for an orientation preserving isometry.
    '''
    for c in M.dual_curves(max_segments=500):
        P=M.drill(c)
        for d in N.dual_curves(max_segments=500):
            Q=N.drill(d)
            for isom in better_is_isometric_to(P,Q,return_isometries=True,index=index,try_hard=try_hard):
                if isom.extends_to_link():
                    if isom.cusp_maps()[0].determinant()==1:
                        return True
    return False

In [13]:
orientation_preserving_isometric(K,F)

True

In [15]:
K.dehn_fill((0,0))
K.identify()

[s912(0,0), 6_3(0,0), K6_43(0,0), K6a1(0,0)]

In [16]:
F.dehn_fill((0,0))
F.identify()

[]

In [17]:
F.simplify()

In [18]:
F.num_tetrahedra()

11